<a href="https://colab.research.google.com/github/Ronaksh21/Panaramic-Image/blob/master/Panaroma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Project Aims at connecting two or more than two images into a single Panaromic Image and It uses SIFT keypoints and determines the excludes the outliers and then uses homography for joining two images.

In [ ]:
pip install opencv-python==3.4.2.16

     |████████████████████████████████| 25.0MB 129kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


In [ ]:
pip install opencv-contrib-python==3.4.2.16


     |████████████████████████████████| 30.6MB 102kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [ ]:
import cv2
import random
import numpy as np
from numpy.linalg import inv
from PIL import Image
import os, os.path


In [ ]:
  
def reading(path):
    img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
    
    if(img is not None):
        print("Image ready for processing")
        return img
    else:
        print("Image cannot be read")

In [ ]:
#calculating the sift paramaterrs
def siftfeatures(img):
    #sift = cv2.SIFT()
    sift = cv2.xfeatures2d.SIFT_create(nfeatures=300)
    kp, des = sift.detectAndCompute(img, None)
    img = cv2.drawKeypoints(img, kp, outImage = None, color=(255,0,0))
    
    cv2.imwrite('keypoints.png',img)
    return (kp ,des)

In [ ]:
def matcher(k1,d1,k2,d2):
  #list1=[][]
  #k1=np.reshape(k1,((np.shape(k1))[0]),1)
  #k2=np.reshape(k2,((np.shape(k2))[0]),1)
  [a]=np.shape(k1)
  [c]=np.shape(k2)
  k1=np.reshape(k1,(a,1))
  k2=np.reshape(k2,(c,1))
  list1 = np.zeros([(np.shape(d1)[0]), (np.shape(d2)[0])]) 
  sum=0
  for i in range((np.shape(d1))[0]):
    for j in range((np.shape(d2))[0]):
      for k in range((np.shape(d1))[1]):
        sum=sum+((d1[i][k]-d2[j][k])**2)

      #sum=np.sqrt(sum)
      list1[i][j]=sum
      sum=0
  max=np.amax(list1)
  print(max)
  
  #for i in range((np.shape(list1))[0]):
  #  for j in range((np.shape(list1)[1])):
  #    list1[i][j]=list1[i][j]/max
  list2=[]
  list1=np.divide(list1,max)
  
  for i in range((np.shape(list1))[0]):
    for j in range((np.shape(list1))[1]):
      if((list1[i][j])<=0.018):
        list2.append((k1[i][0],k2[j][0]))
        

     
  #return list2

  [s,t]=np.shape(list2)

  listxy=[]

  
  for i in range(s):
    [x1,y1]=list2[i][0].pt
    [x2,y2]=list2[i][1].pt
    

    listxy.append([x1, y1, x2, y2]) #Returning matching pixels
  return listxy    #final matched pixels....go to ransac now
  


In [ ]:
def ransac(listxy,th):
  a=len(listxy)
  maxlier=[]
  finalh=None
  for i in range(1200):
    cord1=listxy[random.randint(0,a-1)]
    cord2=listxy[random.randint(0,a-1)]
    cord3=listxy[random.randint(0,a-1)]
    cord4=listxy[random.randint(0,a-1)]

    cord=[[cord1],[cord2],[cord3],[cord4]]
    cord=np.reshape(cord,(4,4))
   
    h=Homo(cord)
    

    inlier=[]
    for c1 in cord:
      d=Distance(c1,h)
      if(d<5):
        inlier.append(c1)

    if(len(inlier)>len(maxlier)):
      maxlier=inlier
      finalh=h
    count=len(cord)*th
    
    if(len(maxlier)>(count)):
      break

  return finalh,maxlier 






In [ ]:
def Homo(cord):

  m1=[]
  m2=[]
  for c in cord:
   
    p1=[c[0],c[1],1]
    p2=[c[2],c[3],1]
    
    l1=[-p1[0],-p1[1],-1,0,0,0,(p1[0]*p2[0]),(p1[1]*p2[0]),p2[0]]
    l2=[0,0,0,-p1[0],-p2[1],-1,(p1[0]*p2[1]),(p1[1]*p2[1]),p2[1]]
    m1.append(l1)
    m1.append(l2)

    m2.append(p2[0])
    m2.append(p2[1])
  l9=[0,0,0,0,0,0,0,0,1]
  m1.append(l9)
  m2.append(1)
  m2=np.reshape(m2,(1,9))
  

  h1=np.matmul(np.matmul(m2,np.transpose(m1)), inv(np.matmul(np.transpose(m1),m1)))
  
  
  h=np.reshape(h1,(3,3))
  
  return h






In [ ]:
def Distance(cordinate,h):

  p1=[cordinate[0],cordinate[1],1]
  p2=[cordinate[2],cordinate[3],1]
  #print(np.shape(p1))
  p1=np.reshape(p1,(3,1))
  p2=np.reshape(p2,(3,1))
  
  expected2=np.matmul(h,p1)
  #print(expected2)
  expected2=np.divide(expected2,expected2[2])
  error=p2-expected2
  
  d=np.linalg.norm(error)
  print(d)
  return d

In [ ]:
def stitch(img1,img2,h):
  y,x=np.nonzero(img1)
  img=img1[np.min(y):np.max(y),np.min(x):np.max(x)]

  stitchimg=img_new=cv2.warpPerspective(img2,h,(img.shape[1]+img2.shape[1],img.shape[0]))
  k=np.nonzero(img)
  stitchimg[k]=img[k]
  return stitchimg

In [ ]:
def main():
  
  threshold=0.6
  ###################################################
  #change the path to the folder containing the images
  ####################################################
  path="/content/sample_data/images/"
  arr = os.listdir(path)
  
  image=None
  for i in range(len(arr)):
    if((i+1)<len(arr)):
      if(image is not None):
        img1=image
        img2=reading(path+str(arr[i]))
      else:
        img1=reading(path+str(arr[i]))
        img2=reading(path+str(arr[i+1]))
      [k1,d1]=siftfeatures(img1)
      [k2,d2]=siftfeatures(img2)


      list2=matcher(k1,d1,k2,d2)
     

      finalh,inliers = ransac(list2, threshold)

      print("final h", finalh)
      print("total inliers", len(inliers))

      #img_new=cv2.warpPerspective(img1,finalh,(img1.shape[1]+img2.shape[1],img1.shape[0]))
      image=stitch(img1,img2,finalh)
      
      cv2.imwrite('resultant_image.jpg',image)





  return 0
main()

Streaming output truncated to the last 5000 lines.
95.17446553568148
76.79061134356537
182.3420026547739
735.4431512472678
731.7705151588547
242.73883717717268
38.08461651826827
22.31254639920281
128.15594703811334
21.11711341164565
31.162952272246198
0.7588636884434999
2.4635052625391
15.376146340711037
38.24357029021803
45.37788692958947
6.700574304088617
134.96906212946917
156.55590637924658
226.38053035971726
152.32311046062574
54.9160746611247
3.8790506309201906
16.29011523548489
430.20518231194035
18.330949839263322
18.056105822164547
19.29908302222165
308.4879100654901
79.15097298195165
9.890810644774172
0.28966709615748215
0.01174452754133233
198.49117194772614
101.14097605444317
50.446732545105085
12.013934502508647
212.15509201566115
60.957892631004206
116.514745352366
184.53067913364228
31.11858284634619
57.6926349801221
488.545632984165
69.18353005046426
0.05696695821925993
86.3209301999784
85.66695791724301
0.4553437459591253
21.947100175223323
28.220357758067276
516.29624

LinAlgError: ignored